In [ ]:
# coding:utf-8

import requests
import csv
from lxml import html

url='https://zz.lianjia.com/ershoufang/pg{page}/'  #目标网页

if __name__ == '__main__':
    start_page=0
    end_page=50                                   #爬取终止页数
    with open('zzhousing.csv','w',newline='') as f:   #以读写模式打开CSV
        csv_writer=csv.writer(f)
        print('start.....')
        while start_page<=end_page:                #循环页数终止判断
            start_page+=1
            print('get:{0}'.format(url.format(page=start_page)))
            response=requests.get(url.format(page=start_page))
            tree = html.fromstring(response.text)
            el = tree.xpath('//div[@class="info clear"]')
            for house in el:                       #遍历每个房产项目
                house_title = house.xpath('div[@class="title"]/a/text()') #名称
                house_addr = house.xpath('div[@class="flood"]/div[@class="positionInfo"]/a/text()') #地址
                house_totalprice = house.xpath('div[@class="priceInfo"]/div[@class="totalPrice"]/span/text()') #单价
                house_unitprice = house.xpath('div[@class="priceInfo"]/div[@class="unitPrice"]/span/text()')   #总价
                house_url = house.xpath('div[@class="title"]/a/@href')   #详情页url网址
                #整理数据
                house_title = ''.join(house_title)
                house_addrs = ''.join(house_addr)
                house_totalprice = ''.join(house_totalprice)
                house_unitprice = ''.join(house_unitprice)
                house_url = ''.join(house_url)

                #数据写入CSV
                csv_writer.writerow([house_title,house_addr, house_unitprice, house_url, house_totalprice+str('万')])
        print('end.....')

In [6]:
# coding:utf-8

import json
from urllib.request import urlopen, quote
import csv

def getlnglat(address):
    url = 'https://api.map.baidu.com/geocoding/v3/'
    output = 'json'
    ak = 'I0rw9y9QZt2aXKdb0xP5PbD0l5e5Q1AT'                  #在这里填写你的密匙
    add = quote(str('郑州市') + address)    #由于本文城市变量为中文，为防止乱码，先用quote进行编码
    uri = url + '?' + 'address=' + add  + '&output=' + output + '&ak=' + ak +'&callback=showLocation'   #传参指定格式，见API文档
    req = urlopen(uri)
    res = req.read().decode() #将其他编码的字符串解码成unicode
    temp = json.loads(res) #对json数据进行解析
    return temp



In [ ]:
file = open(r'/content/point.json','w',encoding='utf-8') #建立json数据文件
with open(r'/content/zzhousing_.csv', 'r') as csvfile: #打开上一步爬取出来csv保存地址
    reader = csv.reader(csvfile)
    for line in reader: #读取csv里的数据
        try:
            # 忽略第一行
            # if reader.line_num == 1: #由于第一行为变量名称，故忽略掉
            #    continue
            # line是个list，取得所有需要的值
            b = line[1].strip()  # 将第二列addr读取出来并清除不需要字符
            c = line[2].strip('元/平')  # 将第三列price读取出来并清除不需要字符
            lng = getlnglat(b)['result']['location']['lng']  # 采用构造的函数来获取经度
            lat = getlnglat(b)['result']['location']['lat']  # 获取纬度
            # str_temp = '{"lat":' + str(lat) + ',"lng":' + str(lng) + ',"count":' + str(c) +'},'
            str_temp = '{"name":"' + str(b) + '","value":[' + str(lng) + ',' + str(lat) + ',' + str(c) + ']},'
            # print(str_temp) #也可以通过打印出来，把数据copy到百度热力地图api的相应位置上
            file.write(str_temp)  # 写入文档
        except:
            print("something wrong about city:" + str(b))   #转换失败即打印错误项目
file.close() #保存